# Exercise 2.5 Task

## 01 - 02. Import Libraries and Data

In [ ]:
# Import Libraries

import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Import Data

df = pd.read_csv('newyork_data.csv', index_col = 0)

## 03. Data Preprocessing

In [ ]:
df.head(10)

In [ ]:
# Create a value column and group by start and end station 

df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()

In [ ]:
df_group

In [ ]:
print(df_group['value'].sum())
print(df.shape)

In [ ]:
df_group['value'].describe()

In [ ]:
# Rename column

df_group.rename(columns = {'value': 'trips'}, inplace = True)

In [ ]:
df_group

In [ ]:
df_group['trips'].describe()

### Exploratory Analysis for Trips

In [ ]:
df_group['trips'].hist(bins = 100)
plt.show()

In [ ]:
df_group['trips'].sort_index().value_counts()

In [ ]:
df_group.to_csv('newyork_stationgroupby_final.csv')

In [ ]:
df_group.head()

In [ ]:
df.head()

### Merge Trip DataFrame with Main DataFrame

In [ ]:
df_m = df.merge(
    df_group,
    how='left',
    on=['start_station_name', 'end_station_name']
)

In [ ]:
df_m

In [ ]:
print(df_m['trips'].sum())
print(df_m.shape)

In [ ]:
df_m.head()

### Create smaller subset of df 

In [ ]:
df_final = df_m[[
    'start_station_name',
    'end_station_name',
    'start_lat',
    'start_lng',
    'end_lat',
    'end_lng',
    'trips'
]].drop_duplicates(subset=['start_station_name', 'end_station_name'])

In [ ]:
df_final.head()

## 04. Kepler.gl map

In [ ]:
from ipywidgets import IntSlider
IntSlider()

In [ ]:
df_final.dtypes

In [ ]:
df_final.head()

In [ ]:
# Looking for missing values

df_final.isnull().sum()

#### Dataframe clean up

In [ ]:
# Replace NaNs in station name with an empty string

df_final['start_station_name'] = df_final['start_station_name'].fillna('Unknown Start Station')

In [ ]:
# Replace NaNs in station name with an empty string

df_final['end_station_name'] = df_final['end_station_name'].fillna('Unknown End Station')

In [ ]:
df_final.columns

In [ ]:
# Looking for missing values

df_final.isnull().sum()

In [ ]:
# Handling missing values in latitiude and longitude. Since we need those to properly map, I'm creating a new df without these

df_clean_final = df_final.replace([np.inf, -np.inf], np.nan)

In [ ]:
# Drop rows missing coordinates

df_clean_final = df_final.dropna(subset=['end_lat', 'end_lng'])

In [ ]:
# Looking for missing values

df_clean_final.isnull().sum()

In [ ]:
# Drop rows missing trips

df_clean_final = df_final.dropna(subset=['trips'])

In [ ]:
# Looking for missing values

df_clean_final.isnull().sum()

In [ ]:
print(df_clean_final['trips'].sum())
print(df_clean_final.shape)

#### Plotting the map

In [ ]:
df_clean_final['trips'].value_counts()

In [ ]:
df_clean_final.head()

In [ ]:
# Create KeplerGl instance

m = KeplerGl(height = 700, data={"data_1": df_clean_final})
m

### Updates made to map:
#### For the Start Filter: I removed color based on "Trips" filter. I also changed the start color to purple to differentiate from the stop color, which I changed to teal. 
#### For the Stop Filter: I unhid the end filter and removed color based on trips. 
#### I changed to show the arc of start to end and changed the colors to purple and teal
#### I added filter on trips and changed colors to purple and teal

### Most popular trips made: 
#### The most popular trips are from the start station of Hoboken Terminal - Hudson St and Hudson Pl with the end station of Hoboken Ave at Monmouth St, with the total trips being 5,565.

### Additional Impressions: 
#### There appears to be a lot of single ride trips, all over the city. A large chunk of data have start stations on one side of the Hudson River and end on the other side, which would suggest a lot of riders are using Citi Bikes to get aross the river. The data indicates that a majority of the rides start in Jersey City/Hoboken/New Jersey area and ride across the George Washington Bridge via the dedicated bike path to New York City/Manahattan. 

## 07. Creating a config

In [ ]:
config = m.config

In [ ]:
config

In [ ]:
import json
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [ ]:
m.save_to_html(file_name = 'New York Citi Bikes Bike Trips Aggregated.html', read_only = False, config = config)